In [158]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import re
import yfinance as yf

In [2]:
df = pd.read_csv('Assets\CONVICTIONSUMMARY_270_opt3_best2(1).csv', sep='|')

In [16]:
df2 = pd.DataFrame()
df2 = df.loc[df['TYPE'] == 'MERGED']

Extract Time and Date

In [ ]:
df2["Time"] = df2.apply(lambda row: re.split(" ",row.DATE)[0], axis=1)

df2["DATE"] = df2.apply(lambda row: re.split(" ",row.DATE)[-1],axis=1)

In [18]:
df2.dtypes

DATE             object
ID               object
STOCK            object
SECTOR           object
TYPE             object
CONVICTIONS     float64
Time             object
dtype: object

In [19]:
df2['DATE'] = pd.to_datetime(
                          df2['DATE'],
                          format='%Y-%m-%d')

<ipython-input-19-3660468bee21>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['DATE'] = pd.to_datetime(


In [41]:
df2 = df2.sort_values(by="DATE")

## Adding returns

In [22]:
list_of_companies = df2["STOCK"].unique()

In [23]:
print(list_of_companies)

['SU' 'GGG' 'WGR' ... 'GBX' 'OVTI' 'SNCAF']


In [227]:
start_date = df2.DATE.min()
end_date = df2.DATE.max()

In [228]:
final_date = end_date + pd.DateOffset(months=12)
print(final_date)

2005-07-14 00:00:00


In [153]:

#define the ticker symbol
tickerSymbol = 'SU'

#get data on this ticker
tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
tickerDf = tickerData.history(period='1m', start=start_date, end=final_date)

#see your data
tickerDf["STOCK"] = tickerSymbol
tickerDf

,Open,High,Low,Close,Volume,Dividends,Stock Splits,STOCK
Date,,,,,,,,
2004-02-10,8.120848,8.370671,8.108198,8.332724,1641400,0.0,0,SU
2004-02-11,8.411781,8.452891,8.304262,8.402294,1280800,0.0,0,SU
2004-02-12,8.383319,8.383319,8.187255,8.196742,950400,0.0,0,SU
2004-02-13,8.301099,8.405456,8.073412,8.114522,1313400,0.0,0,SU
2004-02-17,8.285290,8.291614,8.149310,8.212556,1207000,0.0,0,SU
...,...,...,...,...,...,...,...,...
2005-07-07,15.939996,16.131390,15.697562,16.128201,4025600,0.0,0,SU
2005-07-08,16.412095,16.600300,16.163281,16.335537,3939800,0.0,0,SU
2005-07-11,16.125006,16.472706,16.105867,16.447187,2361600,0.0,0,SU


In [207]:
def get_history_data(ticker_symbol, start_date, end_date):
    tickerData = yf.Ticker(tickerSymbol)
    tickerDf = tickerData.history(period='1m', start=start_date, end=end_date)
    tickerDf["STOCK"] = tickerSymbol

    return tickerDf

    

In [208]:
test = get_history_data("SU",start_date, final_date)
print(test)

                 Open       High        Low      Close   Volume  Dividends  \
Date                                                                         
2004-02-10   8.120846   8.370669   8.108196   8.332722  1641400        0.0   
2004-02-11   8.411784   8.452894   8.304265   8.402297  1280800        0.0   
2004-02-12   8.383317   8.383317   8.187253   8.196740   950400        0.0   
2004-02-13   8.301102   8.405459   8.073415   8.114525  1313400        0.0   
2004-02-17   8.285289   8.291613   8.149309   8.212555  1207000        0.0   
...               ...        ...        ...        ...      ...        ...   
2005-07-07  15.939994  16.131388  15.697560  16.128199  4025600        0.0   
2005-07-08  16.412101  16.600306  16.163287  16.335543  3939800        0.0   
2005-07-11  16.125009  16.472710  16.105871  16.447191  2361600        0.0   
2005-07-12  16.644968  16.967149  16.539700  16.750235  2909200        0.0   
2005-07-13  16.750234  16.756614  16.313216  16.424864  3845800 

In [61]:
new_df = pd.merge(df2, tickerDf,  how='left', left_on=['STOCK','DATE'], right_on = ['STOCK','Date'])

In [156]:
def calculate_returns(start_date, df):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MRerturn','12MReturn'])
    
    date1m = start_date + pd.DateOffset(months=1)
    date3m = start_date + pd.DateOffset(months=3)
    date6m = start_date + pd.DateOffset(months=6)
    date12m = start_date +pd.DateOffset(months=12)

    name = df['STOCK']
    current = df.loc[start_date]
    data1m = df.loc[date1m]
    data3m = df.loc[date3m]
    data6m = df.loc[date6m]
    data12m = df.loc[date12m]

    current = float(current['Close'])
    data1m = float(data1m['Close'])
    data3m = float(data3m['Close'])
    data6m = float(data6m['Close'])
    data12m = float(data12m['Close'])

    return1m = (data1m - current)/current
    return3m = (data3m - current)/current
    return6m = (data6m - current)/current
    return12m = (data6m - current)/current


    returns = pd.DataFrame([[start_date,name[0],return1m, return3m, return6m,return12m]], columns=['DATE','STOCK','1MReturn',"3MReturn","6MReturn","12MReturn"])

 

    return returns

In [157]:
test = calculate_returns(start_date, tickerDf)

print(test)

        DATE STOCK  1MReturn  3MReturn  6MReturn  12MReturn
0 2004-02-11    SU  0.017188 -0.083815  0.052226   0.052226


In [190]:
list_of_dates = df2["DATE"].unique()

In [194]:
list_of_dates[0:3]

array(['2004-02-11T00:00:00.000000000', '2004-02-18T00:00:00.000000000',
       '2004-02-25T00:00:00.000000000'], dtype='datetime64[ns]')

In [204]:
def calculate_returns_for_date(list_of_dates,df):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    for d in list_of_dates:
        temp = calculate_returns(pd.to_datetime(d),df)
        
        returns = returns.append(temp, ignore_index=True)
        #returns = pd.concat(returns,temp)
    
    return returns

In [218]:
test = calculate_returns_for_date(list_of_dates[0:3],tickerDf )

In [219]:
test

,DATE,STOCK,1MReturn,3MReturn,6MReturn,12MReturn
0,2004-02-11,SU,0.017188,-0.083815,0.052226,0.052226
1,2004-02-18,SU,0.070979,-0.078703,0.077539,0.077539
2,2004-02-25,SU,0.038139,-0.044912,0.088722,0.088722


In [230]:
def calculate_returns_for_companies(list_of_dates,list_of_companies,start_date, end_date):
    returns = pd.DataFrame(columns=['DATE','STOCK','1MReturn','3MReturn','6MReturn','12MReturn'])
    for c in list_of_companies:
        data = get_history_data(c,start_date, end_date)
        print(c)
        returns_for_company = calculate_returns_for_date(list_of_dates,data)
        returns = returns.append(returns_for_company, ignore_index=True)
        
    return returns
    

In [231]:
test = calculate_returns_for_companies(list_of_dates, list_of_companies,start_date, end_date)

SU


KeyError: Timestamp('2004-08-11 00:00:00')

In [232]:
test = get_history_data("GGG",start_date,final_date)

In [243]:
test

,Open,High,Low,Close,Volume,Dividends,Stock Splits,STOCK
Date,,,,,,,,
2004-02-10,8.120846,8.370669,8.108196,8.332722,1641400,0.0,0,SU
2004-02-11,8.411781,8.452891,8.304262,8.402294,1280800,0.0,0,SU
2004-02-12,8.383319,8.383319,8.187255,8.196742,950400,0.0,0,SU
2004-02-13,8.301101,8.405458,8.073414,8.114524,1313400,0.0,0,SU
2004-02-17,8.285288,8.291612,8.149308,8.212554,1207000,0.0,0,SU
...,...,...,...,...,...,...,...,...
2005-07-07,15.939996,16.131390,15.697562,16.128201,4025600,0.0,0,SU
2005-07-08,16.412103,16.600308,16.163289,16.335545,3939800,0.0,0,SU
2005-07-11,16.125002,16.472702,16.105863,16.447184,2361600,0.0,0,SU


In [225]:
list_of_companies

array(['SU', 'GGG', 'WGR', ..., 'GBX', 'OVTI', 'SNCAF'], dtype=object)

In [245]:
def get_return_on_date(date,df):
  try:
    x = df.loc[date]
    x = float(x['Close'])
  except:
    x = 0.0
  return x

In [246]:
print(get_return_on_date("2004-02-11",test))

8.402294158935547
